# 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install hanja   # 한자를 한글로 변환하기 위한 패키지
!pip install nltk   # 불용어 제거를 위한 패키지
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git   # 띄어쓰기 바로 잡기 위한 패키지
!pip install git+https://github.com/ssut/py-hanspell.git   # 오타 및 맞춤법을 수정해주는 패키지

# 라이브러리 호출

In [1]:
# 필요한 라이브러리 호출
import pandas as pd
import re
import hanja
from hanspell import spell_checker
from nltk.corpus import stopwords
from pykospacing import Spacing

# 불용어 목록 파일 생성

In [2]:
# NLTK 불용어 다운로드
import nltk
nltk.download('stopwords')
# stopwords 폴더 안에 korean 불용어 파일 만들고 노션에 있는거 복붙해서 넣기

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gustj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# 코랩 사용할 경우 파일 옮기는 방법

In [ ]:
# 주피터가 아니라 코랩 쓰는 경우에만 참고
#지연) 구글 코랩으로 작성해서 직접 생성한 korean파일을 옮기는 코드가 필요했음.. 주피터로 하면 필요없을듯!
# import shutil

# # 파일 경로
# src_path = '/content/korean'
# dest_path = '/root/nltk_data/corpora/stopwords'

# # 파일 이동
# shutil.move(src_path, dest_path)

# 전처리 함수 정의

In [ ]:
# 정규표현식 테스트
patterns = [
    r'<.*?>|\[.*?\]|【.*?】|\{.*?\}',
    r'[^가-힣0-9\s◆◇▲▼]'
]

text = '안녕하세요! [안녕하세요] (안녕하세요★☆)'
for pattern in patterns:
    text = re.sub(pattern, '', text)
print(text)

In [ ]:
text1 = '안녕하세요.이것은 테스트 문장입니다...그렇습니다.'
text2 = '안녕하세요.. 이것은 테스트 문장입니다. 그렇습니다.'

def split_text(article):   # split하는 함수 정의
    sentences = re.split(r'(?<!\.)\.(?!\.)', article)  # 마침표를 기준으로 문장 나누기, 단 마침표가 2개 이상 연달아 있는 경우 제외
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]   # 공백 제거 및 빈 문장 필터링
    return sentences

print(split_text(text1))
print('=====')
print(split_text(text2))

In [7]:
# 데이터 클렌징 함수 정의
def clean_text(text):

    # 0. 한자를 한글로 변환
    # text = re.sub(r'\([^()]*\)', '', text)  # 괄호 안의 한자는 삭제
    text = hanja.translate(text, 'substitution')  # 그 외 한자는 한글로 변환

    # 1. 첫 번째 문장에서 `◆...◆` 패턴 제거
    sentences = text.split('. ')  # 마침표와 공백을 기준으로 문장을 나누기
    if sentences:  # 문장이 존재할 경우
        sentences[0] = re.sub(r'◆.*?◆', '', sentences[0])  # 첫 번째 문장에서 패턴 제거
    text = '. '.join(sentences)  # 문장들을 다시 결합

    # 2. 제거할 단어 및 패턴 목록
    # 기사 전체 제거, 패턴 제거 순으로 함수 실행하기
    patterns = [
        r'[^.]*※[^.]*\.',            # '※'를 포함하는 문장 전체를 제거
        r'[^.]*\[사설\][^.]*\.',      # '[사설]'을 포함하는 문장 전체를 제거
        r'[^.]*마켓PRO[^.]*\.',       # '마켓PRO'를 포함하는 문장 전체를 제거
        r'[^.]*한경유레카[^.]*\.',   # '한경유레카'를 포함하는 문장 전체를 제거
        r'[^.]*이 기사는[^.]*\.',    # '이 기사는'을 포함하는 문장 전체를 제거
        r'[^.]*@[^.]*\.',            # '@'를 포함하는 문장 전체를 제거 / 이메일을 제거하기 위함
        r'[^.]*#장면[^.]*\.',          # '#장면'을 포함하는 문장 전체를 제거
        r'[^.]*\[기자\][^.]*\.',      # '[기자]'를 포함하는 문장 전체를 제거
        r'[^.]*\[저작권\][^.]*\.',    # '[저작권]'을 포함하는 문장 전체를 제거
        r'[^.]*\[퀴즈\][^.]*\.',       # '[퀴즈]'를 포함하는 문장 전체를 제거
        r'[^.]*<용 어>[^.]*\.',
        r'<.*?>|\[.*?\]|【.*?】|\{.*?\}', # ()제외, 온갖 종류의 괄호와 그 안의 내용 전부 제거
        
        r'< 이 기사는 기획 PR 기사 입니다\. >',
        r'< 이 기사는 BizⓝCEO 기획특별판 입니다 >',
        r'[^.?!]*?한경닷컴[^.?!]*?[.?!]',
        r'[^.?!]*?www\.[^.?!]*?([.?!]|$)',
        r'▶ 삼성미술품 \d+만\d+천점 수조원대 이르러…\d+억원 \'행복한 눈물\'은 어디에',
        r'▶ 에버랜드 압수수색',
        r'▶ 삼성특검, 이순동 사장 소환',
        r'▶ 삼성 에버랜드 압수수색 이틀째…긴장 고조',
        r'▶ 특검, \`삼성 전략기획실\' 임원 첫 소환조사',
        r'무단 전재 및 재배포 금지',
        r'한경닷컴\[한경\+ 구독신청\] \[기사구매\] \[모바일앱\]',
        r'ⓒ \'성공을 부르는 습관\' 한경닷컴, 무단 전재 및 재배포 금지',
        r'\(사진\)',
        r'\(가나다순\)',
        r'\(\d{3}\)\d{3}-\d{4}',   # 전화번호
        r'☎',
        r'\s?\w{3}\s*기자',   # ooo 기자
        r'그래픽=',
        r'사진=',
        r'▲대한제강=오형근에서 오치훈으로 대표이사 변경',
        r'[^.!?]*◆특별취재팀[^.!?]*[.!?]',
        r'\[기사보다 빠른 주식정보 , 슈퍼개미 APP\]',
        r'■ SSDsolid state drive',
        r'한경\+는 PC·폰·태블릿에서 읽을 수 있는 한경 디지털 신문입니다구독 plus\.hankyung\.com 문의',
        r'한경부동산',
        r'뉴스룸',
        r'한국경제TV',
        r'최근 애널리스트 분석의견',
        r'주체별 매매동향',
        r'기사제보 및 보도자료',
        r'뉴스래빗 페이스북 facebook\.com/newslabit',
        r'#믿고 보는 #기자 \'한경 기자 코너\'',
        r'▶ 네이버에서 한국경제 뉴스를 받아보세요',
        r'▶ 한경닷컴 바로가기',
        r'▶ 모바일한경 구독신청',
        r'▶ 경제지 네이버 구독 첫 400만, 한국경제 받아보세요',
        r'▶ 한경 고품격 뉴스레터, 원클릭으로 구독하세요',
        r'▶ 한국경제신문과 WSJ, 모바일한경으로 보세요',
        r'▶ 국내 최고의 \'재테크 전문가\'들과 만나는 공간',
        r'▶ 한국경제앱 다운받고 \‘암호화폐\’ 받아가세요',
        r'▶ 한국경제 구독신청-구독료 10% 암호화폐 적립',
        r'ⓒ 한국경제 & hankyung\.com, 무단전재 및 재배포 금지',
        r'THE WALL STREET JOURNAL 한경 독점제휴',
        r'이 기사는 한국경제신문과 금융 AI 전문기업 씽크풀이 공동 개발한 기사 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다\.',
        r'\‘와우밴드\’ 앱을 설치한 후 종목 상담을 신청하면 와우넷 주간수익률 베스트파트너가 자세한 종목 진단을 해 드립니다\.',
    ]

    # 단어 및 패턴을 제거하는 함수
    def remove_patterns(text):
        for pattern in patterns:
            text = re.sub(pattern, '', text)
        return text

    # 패턴 제거
    # text = remove_article(text)
    text = remove_patterns(text)
    
    # 3. 문장 단위로 split하여 리스트에 저장
    def split_text(article):   # split하는 함수 정의
        sentences = re.split(r'(?<!\.)\.(?!\.)', article)  # 마침표를 기준으로 문장 나누기, 단 마침표가 2개 이상 연달아 있는 경우 제외
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]   # 공백 제거 및 빈 문장 필터링
        return sentences

    # 각 문장들을 요소로 갖는 리스트인 text_list
    text_list = split_text(text)

    # 4. 특수 문자 제거 (◆, ◇, ▲, ▼ 제외)
    text_list = [re.sub(r'[^가-힣0-9\s◆◇▲▼]', '', s) for s in text_list]  # 한글과 숫자, 공백, 그리고 ◆, ◇, ▲, ▼만 남김

    # 5. 띄어쓰기 교정
    spacing = Spacing()
    text_list = [spacing(s) for s in text_list]

    # 6. 오타 수정
    def correct_text(sentence):
        result = spell_checker.check(sentence)
        return result.checked   # 수정된 문장 반환
    # 직접 spell_checker 라이브러리 코드 열어서 payload의 key값에 passportKey와 _callback 추가, value는 네이버 맞춤법 검사기 개발자 도구 참고하기
    # data 변수도 바꿔줘야함 -> 자세한건 노션에 서술

    text_list = [correct_text(s) for s in text_list]

    #아래 오류가 났을 때의 코드 -> API 응답이 뜨지 않고 오류 발생이라고 뜨면 라이브러리 코드를 수정 안했거나 잘못한 것
    '''
    def correct_typos(sentence):
        if not isinstance(sentence, str):
            print("입력값이 문자열이 아닙니다.")
            return sentence  # 문자열이 아닐 경우 원본 반환
        try:
            result = spell_checker.check(sentence)
            if not result.result:
                print(f"수정 실패: {result.original}")
                return sentence  # 수정 실패 시 원본 문장 반환
            return result.checked  # 수정된 문장 반환
        except Exception as e:
            print(f"오류 발생: {e}")
            return sentence  # 오류 발생 시 원본 문장 반환

    corrected_text_list = []
    for sentence in text_list:
        try:
            result = spell_checker.check(sentence)
            print(f"API 응답: {result}")  # API 응답 확인
            corrected_text_list.append(result.checked)
        except Exception as e:
            print(f"오류 발생: {e}")
            corrected_text_list.append(sentence)  # 오류 발생 시 원본 문장 반환

    for original, corrected in zip(text_list, corrected_text_list):
        print(f"원본: {original}\n수정: {corrected}\n")
    '''

    # 7. 불용어 제거
    stop_words = set(stopwords.words('korean'))   # korean 불용어 파일 만들어져 있어야 함
    def remove_stopwords(text):   # 불용어 제거 함수 정의
        return ' '.join([word for word in text.split() if word not in stop_words])
    text_list = [remove_stopwords(s) for s in text_list]

    # 8. 분리된 문장들 다시 하나의 텍스트로 합치기
    text = ' '.join(text_list)   # 다시 하나의 문자열로 합쳐진걸 원하면 return text 하면 됨

    return text

# 특정 연도 csv 파일 하나만 돌릴 때 아래 코드 사용

In [ ]:
import time

# CSV 파일을 데이터프레임으로 불러오기
start = time.time()
df = pd.read_csv('2005_한국경제_test.csv')

# 데이터 정제 ) null값과 중복 제거
df.dropna(subset=['body'], inplace=True)
df.drop_duplicates(inplace=True)

# 패턴 목록 정의
patterns = [
    r'<표>',
    r'\[표\]',
    r'\[포토\]',
    r'무신불립',
]

# 패턴이 포함된 행을 삭제하기 위한 조건식 작성
condition1 = df['title'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
df = df[~condition1]
condition2 = df['body'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
df = df[~condition2]

# 'title'과 'body' 컬럼에 클렌징 함수 적용
df['cleaned_title'] = df['title'].apply(clean_text)
df['cleaned_body'] = df['body'].apply(clean_text)

# 결과 확인
print(df[['body', 'cleaned_body']].head())

# 클렌징된 데이터프레임 저장 (원하는 파일명으로 변경)
df.to_csv('cleaned_2005_한국경제_test.csv', index=False, encoding='utf-8-sig')

end = time.time()
print(end - start)

# 여러 연도 csv 파일 반복문으로 돌릴 때 아래 코드 사용

In [ ]:
# 파일 경로 및 범위 설정
start_year = 2005
end_year = 2024

# 모든 연도의 CSV 파일을 처리
for year in range(start_year, end_year + 1):
    filename = f'/{year}_한국경제.csv'

    if os.path.exists(filename):
        print(f"Processing file: {filename}")

        # CSV 파일을 데이터프레임으로 불러오기
        df = pd.read_csv(filename)

        df.dropna(subset=['body'], inplace=True)
        df.drop_duplicates(inplace=True)

        # 패턴 목록 정의
        patterns = [
            r'<표>',
            r'\[표\]',
            r'\[포토\]',
            r'무신불립',
        ]

        # 패턴이 포함된 행을 삭제하기 위한 조건식 작성
        condition1 = df['title'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
        df = df[~condition1]
        condition2 = df['body'].apply(lambda text: any(re.search(pattern, text) for pattern in patterns))
        df = df[~condition2]

        # 'body'와 'title' 컬럼에 클렌징 함수 적용
        df['cleaned_body'] = df['body'].apply(clean_text)
        df['cleaned_title'] = df['title'].apply(clean_text)

        # 클렌징된 데이터프레임 저장
        output_filename = f'cleaned_{year}_한국경제.csv'  
        df.to_csv(output_filename, index=False)

        print(f"Saved cleaned file: {output_filename}")
        
    else:
        print(f"File not found: {filename}")